In [1]:
import numpy as np
import pandas as pd
import os 

In [33]:
def GetPdDataFrame(reponame, colnames=[]):
    fnames = [os.path.join(reponame, x) for x in os.listdir(reponame) if x[0]!='.']
    out = None 
    print 'reading file =', 
    for i, fname in enumerate(fnames):
        print str(i+1)+',',
        if out is None: 
            out = pd.read_csv(fname, sep='\t', header=None, na_filter=False,
                              names=None if (colnames==[]) else colnames, )
        else:
            out = pd.concat((out, pd.read_csv(fname, sep='\t', header=None, na_filter=False,
                                             names=None if (colnames==[]) else colnames)))
    print 
    return out

def PrepareFeatVect(reponame):
    return None

In [37]:
clusterpd = GetPdDataFrame('./training_data/cluster_map/', ['RegionHash', 'RegionID'])
weatherpd = GetPdDataFrame('./training_data/weather_data/', ['time', 'weather', 'temperature', 'pm2.5'])
poipd = GetPdDataFrame('./training_data/poi_data/', [x for x in range(152)])

 reading file = 1,
reading file = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
reading file = 1,


/home/muneebaadil/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:9: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


In [ ]:
orderpd = GetPdDataFrame('./training_data/order_data/', ['OrderID', 'DriverID', 'PassengerID', 
                                                        'StartRegionHash', 'DestRegionHash', 'Price', 'Time'])

reading file = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,